In [1]:
import numpy as np
import caffe
from caffe_io import transform_image
import matplotlib.pyplot as plt
%matplotlib inline
import cv2

In [12]:
image = caffe.io.load_image(
    #'/Users/ecsark/Documents/visualdb/project/wikiart/images/orest-kiprensky_young-gardener-1817.jpg')
    '/Users/ecsark/Documents/visualdb/project/wikiart/images/salvador-dali_still-life-pulpo-y-scorpa.jpg')
    #'/Users/ecsark/Documents/visualdb/project/wikiart/images/david-burliuk_in-the-park.jpg')

In [3]:
caffe_root = '/Users/ecsark/Documents/visualdb/caffe/'
model_weights = caffe_root + 'models/vgg/model.caffemodel'
model_def = caffe_root + 'models/vgg/deploy.prototxt'
net = caffe.Net(model_def, model_weights, caffe.TEST)
net.blobs['data'].reshape(1, 3, 224, 224)

In [4]:
mu = np.array([104.0, 117.0, 123.0])
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))  # move image channels to outermost dimension
transformer.set_mean('data', mu)            # subtract the dataset-mean value in each channel
transformer.set_raw_scale('data', 255)      # rescale from [0, 1] to [0, 255]
transformer.set_channel_swap('data', (2,1,0))  # swap channels from RGB to BGR

In [5]:
def reshape_image(image, new_dim=256, crop_dim=224):
    dsize = (new_dim, new_dim)
    crop_shape = (crop_dim, crop_dim)
    reshaped_img = cv2.resize(image, dsize)
    crop_amt = (new_dim-crop_dim) / 2
    return reshaped_img[crop_amt:crop_amt+crop_dim, crop_amt:crop_amt+crop_dim]

In [14]:
net.blobs['data'].data[...] = transformer.preprocess('data', reshape_image(image, new_dim=256, crop_dim=224))
_ = net.forward(end='fc7')
print net.blobs['fc7'].data[0].tolist()

[-0.49717074632644653, 0.9302692413330078, 0.055869996547698975, -3.4989404678344727, 3.3686137199401855, 1.4227099418640137, -0.7288046479225159, -0.5740044713020325, 1.9106577634811401, 2.217339038848877, -1.9884052276611328, 0.5799435973167419, -0.177598237991333, -0.6472707390785217, -1.6849563121795654, -0.43259239196777344, -0.12173447012901306, -1.9347714185714722, 0.15243607759475708, 1.8095370531082153, 1.3549284934997559, -1.0303962230682373, -1.1322832107543945, -0.2605389952659607, 0.5556442737579346, -1.0480890274047852, 2.106703758239746, -0.797214686870575, -2.741976261138916, -2.2884585857391357, 1.047932505607605, -1.5435194969177246, -2.815676212310791, -0.05565646290779114, -1.440375804901123, 1.7590144872665405, 1.7421863079071045, 1.5934653282165527, -0.7483921051025391, -0.013647198677062988, -1.3792529106140137, -1.5685429573059082, -2.965029716491699, 0.6673038601875305, -1.6939951181411743, -0.28962722420692444, 1.6824533939361572, 0.8263238668441772, -1.701582

In [ ]:
type(fc7)

In [ ]:
import cPickle as pickle
fc7 = pickle.load(open('/Users/ecsark/Documents/visualdb/project/artwork-explorer/experiment/fc7.pk','rb'))

In [ ]:
fc7['features']

In [ ]:
transformed_image = transform_image(image, False, [104,117,123], 256, 224)